<a href="https://colab.research.google.com/github/vanryuji/sqlalchemy/blob/master/sqlalchemy_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sqlalchemy

In [0]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [0]:
# Connecting #

engine = create_engine('sqlite:///:memory:', echo=True)

In [0]:
# Declare mapping

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)


In [5]:
# Create schema

print('before creating, tables:', engine.table_names())
Base.metadata.create_all(engine)
print('after creating, tables:', engine.table_names())

2019-05-09 15:42:55,027 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-09 15:42:55,029 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:42:55,038 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-09 15:42:55,040 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:42:55,042 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-09 15:42:55,044 INFO sqlalchemy.engine.base.Engine ()
before creating, tables: []
2019-05-09 15:42:55,051 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-09 15:42:55,054 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:42:55,058 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-05-09 15:42:55,061 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:42:55,064 INFO sqla

In [6]:
# Creating session

Session = sessionmaker(bind=engine)
session = Session()
print('autocommit:', session.autocommit)

autocommit: False


In [7]:
# Insert

ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
session.commit()
our_user = session.query(User).filter_by(name='ed').all()
print(our_user)

2019-05-09 15:43:08,337 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 15:43:08,342 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-09 15:43:08,346 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-05-09 15:43:08,352 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-09 15:43:08,355 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 15:43:08,359 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-09 15:43:08,362 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [8]:
# Rollback

ed_user = User(name='ed', fullname='Ed Mickey', nickname='edsnickname')
session.add(ed_user)
session.rollback()
users = session.query(User).filter_by(name='ed').all()
print(users)

2019-05-09 15:43:12,378 INFO sqlalchemy.engine.base.Engine ROLLBACK
2019-05-09 15:43:12,381 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 15:43:12,383 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-09 15:43:12,384 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [9]:
# Building a Relationship

class Address(Base):
    __tablename__ = 'addresses'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")
    
Base.metadata.create_all(engine)
print('tables:', engine.table_names())

2019-05-09 15:43:16,052 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-09 15:43:16,058 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:43:16,060 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-05-09 15:43:16,062 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:43:16,065 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-05-09 15:43:16,067 INFO sqlalchemy.engine.base.Engine ()
2019-05-09 15:43:16,068 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-09 15:43:16,070 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-09 15:43:16,071 INFO sqlalchemy.engine.base.Engine ()
tables: ['addresses', 'users']


In [10]:
# Join

jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')
print('jack addresses:', jack.addresses)

jack.addresses = [Address(email_address='jack@google.com'),
                  Address(email_address='j25@yahoo.com')]
session.add(jack)
session.commit()
user = session.query(User).filter_by(name='jack').one()
print(user.addresses)

jack addresses: []
2019-05-09 15:43:25,273 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-09 15:43:25,275 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2019-05-09 15:43:25,279 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-09 15:43:25,281 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 2)
2019-05-09 15:43:25,283 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-09 15:43:25,284 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 2)
2019-05-09 15:43:25,286 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-09 15:43:25,289 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-09 15:43:25,291 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-09